LeetCode SQL Solutions

175. Combine Two Tables

Write a solution to report the first name, last name, city, and state of each person in the Person table. If the address of a personId is not present in the Address table, report null instead.

In [ ]:
# LEFT JOIN
select firstName, lastName, city, state
from person left join address on person.personId = address.personId

In [ ]:
# LEFT JOIN
import pandas as pd
def combine_two_tables(person: pd.DataFrame, address: pd.DataFrame) -> pd.DataFrame:
    df_merge = person.merge(address, on='personId', how='left')
    return df_merge[['firstName', 'lastName', 'city', 'state']]

In [ ]:
# PySpark
result = person.join(address, person.personId == address.personId, 'left')
result = result.select('firstName', 'lastName', 'city', 'state')
result.show()

181. Employees Earning More Than Their Managers

Write a solution to find the employees who earn more than their managers.



In [ ]:
# SELF JOIN
select e1.name as Employee
from employee e1 inner join employee e2 on e1.managerid = e2.id
where e1.salary > e2.salary

In [ ]:
# INNER JOIN
def find_employees(employee: pd.DataFrame) -> pd.DataFrame:
    df_merge = employee.merge(employee, left_on='id', right_on='managerId', how='inner')
    df_filter = df_merge[df_merge['salary_y'] > df_merge['salary_x']]['name_y']
    return pd.DataFrame({'Employee':df_filter})

In [ ]:
# PySpark
from pyspark.sql import function as F

employee_name = (
    employee.alias('e1').join(employee.alias(e2), F.col('e1.managerId') == F.col('e2.id')).filter(F.col('e1.salary') > F.col('e2.salary')).select(F.col('e1.name').alias('Employee'))
                 )

182. Duplicate Emails

Write a solution to report all the duplicate emails. Note that it's guaranteed that the email field is not NULL.



In [ ]:
# GROUP BY
select email
from person
group by email
having count(email) > 1

# CTE, ROW_NUMBER, PARTITION
with cte as (
select email, row_number() over(partition by email order by email) as rn
from person)
select distinct(email)
from cte
where rn>1

# SUB-QUERY, ROW_NUMBER, PARTITION
select email from (
select email, row_number() over(partition by email order by email) as rn
from person) as sq
where rn>1

In [ ]:
# PySpark = GROUBY
email_count = person.groupBy('email').count()
duplicate_emails = email_count.filter(email_count['count']>1).select('email')
return duplicate_emails

# PySpark = ROW_NUMBER
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
window_spec = Window.partitionBy("email").orderBy("email")
person_with_rn = person.withColumn("rn", row_number().over(window_spec))
result = person_with_rn.filter(person_with_rn.rn > 1)
return result.select("email")

In [ ]:
# GROUPBY
def duplicate_emails(person: pd.DataFrame) -> pd.DataFrame:
    df_group = person.groupby('email').agg(email_count=('email', 'count')).reset_index()
    df_filter = df_group[df_group['email_count'] > 1]['email']
    return pd.DataFrame({'Email':df_filter})

# ROW_NUMBER - cumcount()
def duplicate_emails(person: pd.DataFrame) -> pd.DataFrame:
    person['email_count'] = person.groupby('email').cumcount() + 1
    df_filter = person[person['email_count'] > 1]['email'].drop_duplicates()
    return pd.DataFrame({'Email':df_filter})

In [ ]:
# PySpark
person_sorted = person.orderBy('id')
person_unique = person_sorted.dropDuplicates(['email'])
return person_unique

183. Customers Who Never Order

Write a solution to find all customers who never order anything.



In [ ]:
# LEFT JOIN
select customers.name as Customers
from customers left join orders on customers.id = orders.customerId
where customerId is NULL

In [ ]:
# LEFT JOIN & .isnull()
def find_customers(customers: pd.DataFrame, orders: pd.DataFrame) -> pd.DataFrame:
    df_merge = customers.merge(orders, left_on='id', right_on='customerId', how='left')
    df_filter_null = df_merge[df_merge['customerId'].isnull()]['name']
    return pd.DataFrame({'Customers':df_filter_null})

In [ ]:
# PySpark
join_df = customers.join(orders, customers.id == orders.customerId, 'left')
no_orders = join_df.filter(col('customerId'.isNull()))
result = no_order.select(customers['name'].alias('Customers'))

196. Delete Duplicate Emails

Write a solution to delete all duplicate emails, keeping only one unique email with the smallest id.



In [ ]:
# DELETE
delete p1
from person p1, person p2
where p1.email = p2.email and p1.id > p2.id

In [ ]:
# sort_values() & drop_duplicates()
def delete_duplicate_emails(person: pd.DataFrame) -> None:
    person.sort_values(by='id', inplace=True)
    person.drop_duplicates(subset='email', keep='first', inplace=True)

197. Rising Temperature

Write a solution to find all dates' id with higher temperatures compared to its previous dates (yesterday).



In [ ]:
# LEFT JOIN ON DATEDIFF
select w2.id
from weather w1 left join weather w2 on datediff(day, w1.recordDate, w2.recordDate) = 1
where w2.temperature > w1.temperature 

# WINDOW - LAG - PERFORMANCE OPTIMIZED FOR ROW BY ROW OPERATIONS IN REAL WORLD
with cte as (
select id, recorddate, temperature, lag(recorddate, 1) over (order by recorddate) as prevdate, lag(temperature, 1) over (order by recorddate) as prevdaytemp
from weather)
select id
from cte
where temperature > prevdaytemp and datediff(day, prevdate, recorddate) = 1

In [ ]:
# LAG - shift(1)
def rising_temperature(weather: pd.DataFrame) -> pd.DataFrame:
    weather['prev_date'] = weather['recordDate'].shift(1) 
    weather['prev_day_temp'] = weather['temperature'].shift(1)
    df_filter = weather[weather['temperature'] > weather['prev_day_temp']]['id']
    return pd.DataFrame({'id':df_filter})

511. Game Play Analysis I

Write a solution to find the first login date for each player.



In [ ]:
# ROW NUMBER
with cte as (
select *, row_number() over(partition by player_id order by event_date) as rn
from activity)
select player_id, event_date as first_login
from cte
where rn = 1

# MIN
select player_id, min(event_date) as first_login
from activity
group by player_id

577. Employee Bonus

Write a solution to report the name and bonus amount of each employee with a bonus less than 1000.



In [ ]:
# LEFT JOIN
select name, bonus
from employee e left join bonus b on e.empId = b.empId
where bonus < 1000 or bonus is null 

584. Find Customer Referee

Find the names of the customer that are not referred by the customer with id = 2.



In [ ]:
# !=
select name
from customer
where referee_id != 2 or referee_id is null

586. Customer Placing the Largest Number of Orders

Write a solution to find the customer_number for the customer who has placed the largest number of orders.



In [ ]:
# TOP & ORDER BY
select top 1 customer_number
from orders
group by customer_number
order by count(customer_number) desc

595. Big Countries

Write a solution to find the name, population, and area of the big countries.



In [ ]:
# WHERE
select name, population, area
from world
where area >= 3000000 or population >= 25000000

596. Classes More Than 5 Students

Write a solution to find all the classes that have at least five students.



In [ ]:
# GROUP BY AND HAVING
select class
from courses
group by class
having count(class) >= 5

607. Sales Person


Write a solution to find the names of all the salespersons who did not have any orders related to the company with the name "RED".



In [ ]:
# JUST JOIN
SELECT s.name
FROM salesperson s
LEFT JOIN orders o ON s.sales_id = o.sales_id
LEFT JOIN company c ON o.com_id = c.com_id AND c.name = 'RED'
WHERE c.com_id IS NULL;

# CTE
WITH cte AS (
  SELECT s.name
  FROM salesperson s
  LEFT JOIN orders o ON s.sales_id = o.sales_id
  LEFT JOIN company c ON o.com_id = c.com_id
  WHERE c.name = 'RED' AND s.name IS NOT NULL
)
SELECT name
FROM salesperson
WHERE name NOT IN (SELECT name FROM cte);

# SUB QUERY
SELECT s.name
FROM salesperson s
WHERE NOT EXISTS (
  SELECT 1
  FROM orders o
  JOIN company c ON o.com_id = c.com_id
  WHERE o.sales_id = s.sales_id AND c.name = 'RED'
);

610. Triangle Judgement

Report for every three line segments whether they can form a triangle.



In [ ]:
# CASE STATEMENT
select x, y, z, 
case 
when x+y>z and x+z>y and y+z>x then 'Yes'
else 'No'
end as Triangle
from triangle

619. Biggest Single Number

A single number is a number that appeared only once in the MyNumbers table.



In [ ]:
# EXTRACT THE SINGLE NUMBERS FIRST AND THEN GET MAX FROM IT
select max(num) as num
from (
    select num
    from MyNumbers
    group by num
    having count(num) < 2
) as sq

# WITHOUT SUBQUERY
SELECT MAX(num) AS num
FROM MyNumbers
GROUP BY num
HAVING COUNT(num) < 2;

1661. Average Time of Process per Machine

There is a factory website that has several machines each running the same number of processes. Write a solution to find the average time each machine takes to complete a process.

The time to complete a process is the 'end' timestamp minus the 'start' timestamp. The average time is calculated by the total time to complete every process on the machine divided by the number of processes that were run.

The resulting table should have the machine_id along with the average time as processing_time, which should be rounded to 3 decimal places.

In [ ]:
# SELF JOIN
select a.machine_id, ROUND(AVG(b.timestamp - a.timestamp),3) as processing_time
from activity a inner join activity b
on a.machine_id = b.machine_id and a.process_id = b.process_id and a.activity_type = 'start' and b.activity_type = 'end'
group by a.machine_id

# LAG
with cte as (
    select *, lag(timestamp, 1) over(partition by machine_id, process_id order by machine_id) as prev_timestamp
    from activity
)
select machine_id, AVG(timestamp - prev_timestamp) as processing_time
from cte
where activity_type = 'end'
group by machine_id

1280. Students and Examinations

Write a solution to find the number of times each student attended each exam.



In [ ]:
# CROSS JOIN
with cte as (
select *
from students st cross join subjects su)

select c.student_id, c.student_name, c.subject_name, count(e.subject_name) as attended_exams
from cte c left join examinations e on c.student_id = e.student_id and c.subject_name = e.subject_name
group by c.student_id, c.student_name, c.subject_name

570. Managers with at Least 5 Direct Reports

Write a solution to find managers with at least five direct reports.



In [ ]:
# SELF and GROUP BY HAVING
select e2.name
from employee e1 left join employee e2
on e1.managerId = e2.id
group by e2.id, e2.name  # Look closely here
having count(e2.id) >= 5

1934. Confirmation Rate

Write a solution to find the confirmation rate of each user.

In [ ]:
# CASE
select s.user_id, ROUND(AVG(case when c.action = 'confirmed' then 1.0 else 0.0 end), 2) as confirmation_rate
from signups s left join confirmations c
on s.user_id = c.user_id
group by s.user_id

1251. Average Selling Price

Write a solution to find the average selling price for each product. average_price should be rounded to 2 decimal places. If a product does not have any sold units, its average selling price is assumed to be 0.

In [ ]:
# JOIN and JOIN Conditions
select p.product_id, COALESCE(ROUND(SUM(u.units * p.price * 1.0)/SUM(u.units * 1.0),2), 0) as average_price
from prices p left join unitssold u 
on p.product_id = u.product_id and u.purchase_date >= p.start_date and u.purchase_date <= p.end_date
group by p.product_id

1075. Project Employees I

Write an SQL query that reports the average experience years of all the employees for each project, rounded to 2 digits.



In [ ]:
# AVG and GROUP BY
select p.project_id, round(sum(e.experience_years*1.0)/count(*),2) as average_years
from project p left join employee e
on p.employee_id = e.employee_id
group by project_id

1633. Percentage of Users Attended a Contest

Write a solution to find the percentage of the users registered in each contest rounded to two decimals.

Return the result table ordered by percentage in descending order. In case of a tie, order it by contest_id in ascending order.

In [ ]:
# Without Join
SELECT  r.contest_id, ROUND(COUNT(DISTINCT r.user_id) * 100.0 / (SELECT COUNT(DISTINCT user_id) FROM users), 2) AS percentage
FROM register r
GROUP BY r.contest_id
ORDER BY percentage DESC, contest_id ASC;

# Cross Join
select r.contest_id, round(count(distinct r.user_id)*100.0/count(distinct u.user_id), 2) as percentage
from users u 
cross join register r
group by r.contest_id
order by percentage desc, contest_id asc

# Precomputing the unique count
DECLARE @total_users_count INT;
SELECT @total_users_count = COUNT(DISTINCT user_id) FROM users;
SELECT r.contest_id, ROUND(COUNT(DISTINCT r.user_id) * 100.0 / @total_users_count, 2) AS percentage
FROM register r
GROUP BY r.contest_id
ORDER BY percentage DESC, contest_id ASC;

1211. Queries Quality and Percentage

We define query quality as:

The average of the ratio between query rating and its position.

We also define poor query percentage as:

The percentage of all queries with rating less than 3.

Write a solution to find each query_name, the quality and poor_query_percentage.

Both quality and poor_query_percentage should be rounded to 2 decimal places.

In [ ]:
# Case, When there is need to insert select statement to get a new column and group by it, think of if you can use CASE 
SELECT 
query_name,
ROUND(SUM(rating*1.0/position) / COUNT(*),2) as quality,
ROUND(SUM(CASE WHEN rating<3 THEN 1 ELSE 0 END)*100.0 / COUNT(*),2) as poor_query_percentage
FROM queries
GROUP BY query_name

1193. Monthly Transactions I

Write an SQL query to find for each month and country, the number of transactions and their total amount, the number of approved transactions and their total amount.



In [ ]:
# DATE and CASE
select 
FORMAT(trans_date, 'yyyy-MM') as month, 
country,
count(FORMAT(trans_date, 'yyyy-MM')) as trans_count,
sum(case when state = 'approved' then 1 else 0 end) as approved_count,
sum(amount) as trans_total_amount,
sum(case when state = 'approved' then amount else 0 end) as approved_total_amount
from transactions
group by FORMAT(trans_date, 'yyyy-MM'), country

1174. Immediate Food Delivery II

If the customer's preferred delivery date is the same as the order date, then the order is called immediate; otherwise, it is called scheduled.

The first order of a customer is the order with the earliest order date that the customer made. It is guaranteed that a customer has precisely one first order.

In [ ]:
# CTE and CASE
with cte as (
select *, row_number() over(partition by customer_id order by order_date) as rn
from delivery)

SELECT ROUND((sum(CASE WHEN order_date = customer_pref_delivery_date THEN 1.0 ELSE 0.0 END) * 1.0 / COUNT(*))*100, 2) AS immediate_percentage
FROM cte
where rn = 1

550. Game Play Analysis IV

Write a solution to report the fraction of players that logged in again on the day after the day they first logged in, rounded to 2 decimal places. In other words, you need to count the number of players that logged in for at least two consecutive days starting from their first login date, then divide that number by the total number of players.



In [ ]:
# CTE, DATEDIFF
with cte as (
    select player_id, min(event_date) as all_players_min_date
    from activity
    group by player_id
)

select round(count(a.player_id)*1.0/count(c.player_id), 2) as fraction
from cte c left join activity a
on c.player_id = a.player_id and datediff(day, c.all_players_min_date, a.event_date) = 1

1141. User Activity for the Past 30 Days I

Write a solution to find the daily active user count for a period of 30 days ending 2019-07-27 inclusively. A user was active on someday if they made at least one activity on that day.



In [ ]:
# Between is inclusive and you can use dateadd -30
select activity_date as day, count(distinct user_id) as active_users
from activity
where activity_date > dateadd(day, -30, '2019-07-27') and activity_date <= '2019-07-27'
group by activity_date

1070. Product Sales Analysis III

Write a solution to select the product id, year, quantity, and price for the first year of every product sold.



In [ ]:
#Approach 2;  Using CTE
with cte as (
    select product_id, min(year) as min_year
    from sales
    group by product_id
)
select c.product_id, c.min_year as first_year, s.quantity, s.price
from cte c inner join sales s
on c.product_id = s.product_id and c.min_year = s.year


#Approach 2;  Using IN
SELECT product_id, year AS first_year, quantity, price
FROM Sales
WHERE (product_id, year) in (
    SELECT product_id, MIN(year) 
    FROM Sales
    GROUP BY product_id
)

#Approach 3 : Using Except
SELECT product_id, year AS first_year, quantity, price
FROM Sales
EXCEPT
SELECT s1.product_id, s1.year, s1.quantity, s1.price
FROM Sales s1
JOIN Sales s2 ON s1.product_id = s2.product_id AND s1.year > s2.year;

619. Biggest Single Number

A single number is a number that appeared only once in the MyNumbers table.



In [ ]:
# CTE and MAX
with cte as (
    select num, count(num) as cnt
    from mynumbers
    group by num
    having count(num) = 1)
select max(num) as num
from cte

# ORDER BY DESC and TOP 1
SELECT coalesce ((
    SELECT top 1 num
FROM mynumbers
GROUP BY num
HAVING COUNT(*) = 1
ORDER BY num desc), null) as num

Customers Who Bought All Products


Write a solution to report the customer ids from the Customer table that bought all the products in the Product table.



In [ ]:
# Precompute
DECLARE @distinct_products INT;
SELECT @distinct_products = COUNT(product_key) from product

select customer_id
from customer
group by customer_id
having count(distinct product_key) = @distinct_products

1731. The Number of Employees Which Report to Each Employee

For this problem, we will consider a manager an employee who has at least 1 other employee reporting to them.

Write a solution to report the ids and the names of all managers, the number of employees who report directly to them, and the average age of the reports rounded to the nearest integer.



In [ ]:
SELECT e1.employee_id, e1.name, COUNT(e2.employee_id) as reports_count, ROUND(AVG(e2.age*1.0),0) as average_age
FROM employees e1
INNER JOIN employees e2 ON e1.employee_id = e2.reports_to
GROUP BY e1.employee_id, e1.name
ORDER BY employee_id

SQL Notes:

* Finding Duplicate Rows
  * GROUP BY and HAVING COUNT > 1,  
  * CTE, ROW_NUMBER, PARTITION
  * SUBQUERY, ROW_NUMBER, PARTITION

* Finding Duplicate Rows From Two Columns
  * DISTINCT
  
* Customers who never ordered anything
  * SUBQUERY
  * JOIN and Filter NULL
* Decimal Places
  * COL*1.0 instead of casting into decimal
  * cast (col) as decimal(10,2)
  * Cast (col) as float and then ROUND(col, 2)
* Finding Consecutive
  * Use JOIN based on condition col+1 = 1 or col-1=1 and where value is 1
  * LAG and LEAD
* Salesperson who did not have any orders with a company
  * Find the names and then do not in subquery
* Compare with all the items
  * Cross join
* Iff 
  * Update salary set sex = iff(sex=’f’, ‘m’, ‘f’)
  * Use CASE WHEN THEN END
* Count of 2 different columns together
  * Use group by for two different columns and in having put any other third column apart from these 2
  * We don't need to just use the two columns in select to put the inside count.
* Dates:
  * Use MIN and MAX for Dates
  * BETWEEN is Inclusive - (where activity_date between ‘small_date’ and ‘big_date’)
  * datediff(day, today, previousdate)
  * dateadd(day, -1, date)
* HANDLING NULLS
  * Return the specified value IF the expression is NULL, otherwise return the expression:
    * SELECT ISNULL(NULL, 'W3Schools.com');
  * Compare two expressions:
    * SELECT NULLIF(25, 25);
* Comparing two side by side columsn
  * CASE
* Pivot
  * USE PIVOT
  * Or CASE


SQL Commands

* Retrieves specific columns from a table  
  SQL Command: `SELECT column1, column2 FROM table_name`

* Retrieves all columns from a table where a specified condition is met  
  SQL Command: `SELECT * FROM table_name WHERE condition`

* Counts the number of rows for each group of a specified column  
  SQL Command: `SELECT column1, COUNT(*) FROM table_name GROUP BY column1`

* Joins two tables based on a common column and retrieves all columns  
  SQL Command: `SELECT * FROM table1 JOIN table2 ON table1.column = table2.column`

* Retrieves all columns from a table and sorts the results in ascending order  
  SQL Command: `SELECT * FROM table_name ORDER BY column ASC`

* Creates or replaces a temporary view from a table  
  SQL Command: `CREATE OR REPLACE TEMP VIEW view_name AS SELECT * FROM table_name`

* Retrieves all columns from a view where a specified condition is met  
  SQL Command: `SELECT * FROM view_name WHERE condition`

* Inserts new data into specified columns of a table  
  SQL Command: `INSERT INTO table_name (column1, column2) VALUES (value1, value2)`

* Retrieves all columns from a table  
  SQL Command: `SELECT * FROM table_name`

* Counts the total number of rows in a table  
  SQL Command: `SELECT COUNT(*) FROM table_name`

* Retrieves distinct combinations of specified columns from a table  
  SQL Command: `SELECT DISTINCT column1, column2 FROM table_name`

* Calculates the average of a column grouped by another column  
  SQL Command: `SELECT column1, AVG(column2) FROM table_name GROUP BY column1`

* Renames a column in the result set  
  SQL Command: `SELECT column1 AS new_column_name FROM table_name`

* Retrieves the first 10 rows from a table  
  SQL Command: `SELECT * FROM table_name LIMIT 10`

* Calculates the sum of a specified column in a table  
  SQL Command: `SELECT SUM(column1) FROM table_name`

* Retrieves all columns from a table  
  SQL Command: `SELECT * FROM table_name`

* Retrieves rows where a specified column is NULL or NOT NULL  
  SQL Command: `SELECT column1 FROM table_name WHERE column2 IS NULL/NOT NULL`

* Retrieves rows where a column's value is greater than or equal to a specified value  
  SQL Command: `SELECT * FROM table_name WHERE column >= value`

* Retrieves rows where a column's value matches any in a list of values  
  SQL Command: `SELECT * FROM table_name WHERE column IN (value1, value2)`

* Retrieves rows where a column's value is within a specified range  
  SQL Command: `SELECT * FROM table_name WHERE column BETWEEN value1 AND value2`

* Retrieves rows where a column matches a specified pattern  
  SQL Command: `SELECT * FROM table_name WHERE column LIKE 'pattern'`

* Converts a column's data type to a specified type  
  SQL Command: `SELECT CAST(column AS data_type) FROM table_name`


Pandas Commands

* Select a single column from the DataFrame:  
  `df['column1']`

* Select multiple columns:  
  `df[['column1', 'column2']]`

* Filter rows where the condition is true:  
  `df[df['column'] > value]`

* Group by one column and sum values of another column:  
  `df.groupby('column1')['column2'].sum()`

* Group by one column and calculate an aggregate (e.g., mean):  
  `df.groupby('column1').agg({'column2': 'mean'})`

* Merge two DataFrames on a common column:  
  `df1.merge(df2, on='column', how='inner')`

* Sort the DataFrame by a column:  
  `df.sort_values(by='column', ascending=False)`

* Remove duplicate rows based on specific columns:  
  `df.drop_duplicates(subset=['column1', 'column2'])`

* Drop specified columns from the DataFrame:  
  `df.drop(columns=['column1', 'column2'])`

* Rename columns:  
  `df.rename(columns={'old_name': 'new_name'})`

* Replace NaN values with a specified value:  
  `df.fillna(value, inplace=True)`

* Check for null values in each column:  
  `df.isnull().sum()`

* Create a new column based on existing ones:  
  `df['new_column'] = df['column1'] + df['column2']`

* Update values in a new column based on a condition:  
  `df.loc[df['column'] > value, 'new_column'] = 'value'`

* Change the data type of a column:  
  `df['column'].astype('int')`

* Apply a function to each element in a column:  
  `df['column'].apply(lambda x: x * 2)`

* Create a pivot table:  
  `df.pivot_table(index='column1', columns='column2', values='column3')`

* Export DataFrame to a CSV file:  
  `df.to_csv('filename.csv', index=False)`

* Load a CSV file into a DataFrame:  
  `pd.read_csv('filename.csv')`

* Generate summary statistics for numerical columns:  
  `df.describe()`

* Count unique values in a column:  
  `df['column'].value_counts()`

* Randomly select 5 rows from the DataFrame:  
  `df.sample(n=5)`

* Display the first 10 rows of the DataFrame:  
  `df.head(10)`

* Display the last 10 rows of the DataFrame:  
  `df.tail(10)`


PySpark Commands

* **Select specific columns from a DataFrame:**  
  `df.select("column1", "column2")`  

* **Filter rows based on a condition:**  
  `df.filter(condition)`  

* **Group by a column and count rows in each group:**  
  `df.groupBy("column1").count()`  

* **Join two DataFrames on a column:**  
  `df1.join(df2, df1.column == df2.column)`  

* **Sort rows by a column in ascending order:**  
  `df.orderBy("column", ascending=True)`  

* **Create or replace a temporary SQL view from a DataFrame:**  
  `df.createOrReplaceTempView("view_name")`  

* **Run a SQL query on a temporary view:**  
  `spark.sql("SELECT * FROM view_name WHERE condition").show()`  

* **Write a DataFrame to a table in overwrite mode:**  
  `df.write.mode("overwrite").saveAsTable("table_name")`  

* **Read a table into a DataFrame:**  
  `df = spark.read.table("table_name")`  

* **Count the number of rows in a DataFrame:**  
  `df.count()`  

* **Remove duplicate rows based on specified columns:**  
  `df.dropDuplicates(["column1", "column2"])`  

* **Group by a column and compute the average of another column:**  
  `df.groupBy("column1").avg("column2")`  

* **Rename a column in the DataFrame:**  
  `df.withColumnRenamed("column1", "new_column_name")`  

* **Limit the number of rows in the output:**  
  `df.limit(10)`  

* **Evaluate an expression (e.g., sum of a column):**  
  `df.selectExpr("sum(column1)").show()`  

* **Drop specified columns from the DataFrame:**  
  `df.drop("column1", "column2")`  

* **Filter rows where a column is null or not null:**  
  `df.filter(df.column2.isNull()) / df.filter(df.column2.isNotNull())`  

* **Filter rows where a column is greater than or equal to a value:**  
  `df.filter(df.column >= value).show()`  

* **Filter rows where a column's value is in a list of values:**  
  `df.filter(df.column.isin(value1, value2)).show()`  

* **Filter rows where a column's value falls between two values:**  
  `df.filter(df.column.between(value1, value2)).show()`  

* **Filter rows where a column's value matches a pattern:**  
  `df.filter(df.column.like('pattern')).show()`  

* **Cast a column to a specific data type:**  
  `df.selectExpr("CAST(column AS data_type)").show()`  
